In [1]:
import numpy as np
import pandas as pd

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/instadeep-challenge-soai/sequence_esm1_t34_670M_UR100_cls_embeddings/sequence_esm1_t34_670M_UR100_cls_embeddings.csv
/kaggle/input/instadeep-challenge-soai/archive/submissionSample.csv
/kaggle/input/instadeep-challenge-soai/archive/train.csv
/kaggle/input/instadeep-challenge-soai/archive/test.csv


In [2]:
from sklearn.utils import shuffle
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [3]:
train_dataset = "/kaggle/input/instadeep-challenge-soai/archive/train.csv"
test_dataset = "/kaggle/input/instadeep-challenge-soai/archive/test.csv"
df = pd.read_csv(train_dataset)
# df = shuffle(pd.read_csv(train_dataset))
df_test = pd.read_csv(test_dataset)

In [4]:
df.head()

,ID,protein_name,sequence,biological_process,cellular_component,molecular_function
0,train_0,sp|Q9J523|V212_FOWPN,MPQNKVLSFPLPEGTLLEDITKNKWILGKQLGSGGFGLVYQVSCKS...,0,0,1
1,train_1,sp|Q14990|ODFP1_HUMAN,MAALSCLLDSVRRDIKKVDRELRQLRCIDEFSTRCLCDLYMHPYCC...,0,0,1
2,train_2,sp|Q9C994|DTX14_ARATH,MDSAEKGLLVVSDREEVNKKDGFLRETKKLSYIAGPMIAVNSSMYV...,0,0,1
3,train_3,sp|Q58425|CARA_METJA,MEAVLILEDGTILKGKGFGAEKEVFGELVFTTVMTGYVEVLTDPSY...,0,1,0
4,train_4,sp|P78814|SRP2_SCHPO,MSETRLFVGRIPPQATREDMMDFFKGYGQILDCKLMNGFGFVEVED...,0,0,1


In [5]:
df.shape

(33622, 6)

In [6]:
df_test.head()

,ID,protein_name,sequence
0,test_0,sp|O02056|RL4_CAEEL,MAARPLVTVYDEKYEATQSQIRLPAVFRTPIRPDLVSFIADQVRRN...
1,test_1,sp|G4RJZ4|CS3HD_THETK,MSCCAYFRGRDCLQTYEDHITQALEACERLRPYYGRWMEKAFGTAD...
2,test_2,sp|P03109|VL2_BPV1,MSARKRVKRASAYDLYRTCKQAGTCPPDVIPKVEGDTIADKILKFG...
3,test_3,sp|Q61473|SOX17_MOUSE,MSSPDAGYASDDQSQPRSAQPAVMAGLGPCPWAESLSPLGDVKVKG...
4,test_4,sp|P51641|CNTFR_CHICK,MANPVPSACCVVLAAVVVVYAQRHSQQDSHIQYERVGADVTMKCGS...


In [7]:
df_test.shape

(14410, 3)

In [8]:
df.columns

Index(['ID', 'protein_name', 'sequence', 'biological_process',
       'cellular_component', 'molecular_function'],
      dtype='object')

In [9]:
df.set_index('ID', inplace = True)
target = df[df==1].stack().reset_index().drop(0, axis=1)
df.drop(["biological_process","cellular_component","molecular_function"], axis = 1, inplace = True)
df['target'] = target[['level_1']].values
df.reset_index(inplace = True)
df.head()

,ID,protein_name,sequence,target
0,train_0,sp|Q9J523|V212_FOWPN,MPQNKVLSFPLPEGTLLEDITKNKWILGKQLGSGGFGLVYQVSCKS...,molecular_function
1,train_1,sp|Q14990|ODFP1_HUMAN,MAALSCLLDSVRRDIKKVDRELRQLRCIDEFSTRCLCDLYMHPYCC...,molecular_function
2,train_2,sp|Q9C994|DTX14_ARATH,MDSAEKGLLVVSDREEVNKKDGFLRETKKLSYIAGPMIAVNSSMYV...,molecular_function
3,train_3,sp|Q58425|CARA_METJA,MEAVLILEDGTILKGKGFGAEKEVFGELVFTTVMTGYVEVLTDPSY...,cellular_component
4,train_4,sp|P78814|SRP2_SCHPO,MSETRLFVGRIPPQATREDMMDFFKGYGQILDCKLMNGFGFVEVED...,molecular_function


In [10]:
df['target'] = df['target'].replace(["biological_process","cellular_component","molecular_function"],
                                              [0,1,2]).values

In [11]:
embeddings = pd.read_csv('/kaggle/input/instadeep-challenge-soai/sequence_esm1_t34_670M_UR100_cls_embeddings/sequence_esm1_t34_670M_UR100_cls_embeddings.csv')
embeddings.head()

,ID,emb_0,emb_1,emb_2,emb_3,emb_4,emb_5,emb_6,emb_7,emb_8,...,emb_1270,emb_1271,emb_1272,emb_1273,emb_1274,emb_1275,emb_1276,emb_1277,emb_1278,emb_1279
0,test_0,0.910324,-2.170328,10.041729,-0.678233,-1.482925,3.012937,-0.003094,4.098589,1.651323,...,-1.840684,3.354303,-2.792454,4.320904,-1.169091,1.783700,0.909022,-1.176727,3.376020,1.670808
1,test_1,0.052961,-2.283634,8.310371,-0.688876,-2.549909,4.436736,-2.260496,3.848493,2.184023,...,-1.234872,0.903201,-2.600311,3.054049,0.021382,2.002324,-0.153883,-0.273578,2.630812,-2.386319
2,test_10,-0.987252,-2.370148,7.980342,-0.957545,-4.070104,4.300144,-1.012244,5.221393,0.064858,...,-2.143104,3.015208,-3.198480,4.431435,0.253012,1.612991,1.496582,-1.986429,3.795451,-1.674614
3,test_100,2.458594,-1.872591,9.825299,-2.611592,-1.166800,2.984232,0.151903,6.202667,-1.292036,...,-2.919709,2.744383,-2.144281,1.760952,0.243129,-1.766697,0.164476,-1.829063,0.768626,-2.818253
4,test_1000,-0.106690,-3.072514,7.511356,-2.352259,-2.718528,5.221254,-2.282844,5.829901,0.654640,...,-1.801517,2.597255,-3.051652,4.678716,0.739257,1.202128,0.504353,-1.732116,1.001569,-5.333856


In [12]:
embeddings.shape

(48032, 1281)

In [13]:
embeddings.sort_values(by = "ID")

,ID,emb_0,emb_1,emb_2,emb_3,emb_4,emb_5,emb_6,emb_7,emb_8,...,emb_1270,emb_1271,emb_1272,emb_1273,emb_1274,emb_1275,emb_1276,emb_1277,emb_1278,emb_1279
0,test_0,0.910324,-2.170328,10.041729,-0.678233,-1.482925,3.012937,-0.003094,4.098589,1.651323,...,-1.840684,3.354303,-2.792454,4.320904,-1.169091,1.783700,0.909022,-1.176727,3.376020,1.670808
1,test_1,0.052961,-2.283634,8.310371,-0.688876,-2.549909,4.436736,-2.260496,3.848493,2.184023,...,-1.234872,0.903201,-2.600311,3.054049,0.021382,2.002324,-0.153883,-0.273578,2.630812,-2.386319
2,test_10,-0.987252,-2.370148,7.980342,-0.957545,-4.070104,4.300144,-1.012244,5.221393,0.064858,...,-2.143104,3.015208,-3.198480,4.431435,0.253012,1.612991,1.496582,-1.986429,3.795451,-1.674614
3,test_100,2.458594,-1.872591,9.825299,-2.611592,-1.166800,2.984232,0.151903,6.202667,-1.292036,...,-2.919709,2.744383,-2.144281,1.760952,0.243129,-1.766697,0.164476,-1.829063,0.768626,-2.818253
4,test_1000,-0.106690,-3.072514,7.511356,-2.352259,-2.718528,5.221254,-2.282844,5.829901,0.654640,...,-1.801517,2.597255,-3.051652,4.678716,0.739257,1.202128,0.504353,-1.732116,1.001569,-5.333856
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48027,train_9995,0.807301,-2.093418,5.033188,-2.958990,0.397691,5.696020,2.038990,5.385627,1.069396,...,0.322275,3.068091,-1.920059,0.626606,-1.799351,-0.857117,0.257831,-0.683630,4.092725,-1.423994
48028,train_9996,-1.380142,0.344023,5.243223,-2.528724,-0.618228,3.348662,-2.002979,2.793901,0.855594,...,-0.662560,3.135431,-2.758837,1.247987,1.511210,1.767873,0.849789,-3.190919,-0.244494,-1.630793
48029,train_9997,-0.723169,-2.624363,8.035697,-0.557193,-6.043829,4.345010,-0.453897,4.156330,0.316079,...,-2.111323,4.283244,-4.254265,4.223408,-0.477268,1.349393,1.683797,-0.271883,2.759504,-1.300398
48030,train_9998,1.285597,-2.949579,8.993331,-2.806756,-2.216582,3.998766,1.594580,5.928559,0.586525,...,-1.916933,1.387129,-3.212661,1.277672,-0.869022,0.799807,-0.779574,-1.946847,1.305018,-2.391864


In [14]:
embeddings[embeddings['ID'] == 'train_0']

,ID,emb_0,emb_1,emb_2,emb_3,emb_4,emb_5,emb_6,emb_7,emb_8,...,emb_1270,emb_1271,emb_1272,emb_1273,emb_1274,emb_1275,emb_1276,emb_1277,emb_1278,emb_1279
14410,train_0,0.081642,-1.496138,6.191256,0.67927,-1.894954,4.502281,-0.51448,3.705019,-0.492262,...,0.03982,3.243101,-2.772209,2.121812,-0.186634,2.419172,1.625475,-1.370067,3.178513,-1.555542


In [19]:
# copy the data
df_max_scaled = embeddings.drop('ID', axis=1).copy()

# apply normalization techniques
for column in df_max_scaled.columns:
    df_max_scaled[column] = df_max_scaled[column] / df_max_scaled[column].abs().max()
    
# view normalized data
display(df_max_scaled)

,emb_0,emb_1,emb_2,emb_3,emb_4,emb_5,emb_6,emb_7,emb_8,emb_9,...,emb_1270,emb_1271,emb_1272,emb_1273,emb_1274,emb_1275,emb_1276,emb_1277,emb_1278,emb_1279
0,0.163292,-0.214504,0.772762,-0.100303,-0.188323,0.325881,-0.000539,0.468595,0.308145,0.134281,...,-0.321614,0.457562,-0.353930,0.465998,-0.168417,0.260290,0.162296,-0.180027,0.441839,0.294106
1,0.009500,-0.225702,0.639525,-0.101877,-0.323824,0.479880,-0.393520,0.440001,0.407549,0.502149,...,-0.215763,0.123206,-0.329576,0.329371,0.003080,0.292193,-0.027474,-0.041855,0.344309,-0.420055
2,-0.177092,-0.234253,0.614128,-0.141611,-0.516879,0.465106,-0.176217,0.596966,0.012103,-0.108849,...,-0.374454,0.411306,-0.405391,0.477918,0.036448,0.235378,0.267199,-0.303903,0.496732,-0.294776
3,0.441019,-0.185077,0.756106,-0.386226,-0.148177,0.322777,0.026444,0.709156,-0.241100,0.518319,...,-0.510146,0.374362,-0.271777,0.189914,0.035025,-0.257808,0.029366,-0.279828,0.100594,-0.496086
4,-0.019138,-0.303671,0.578037,-0.347874,-0.345237,0.564734,-0.397410,0.666537,0.122159,-0.084848,...,-0.314770,0.354293,-0.386782,0.504587,0.106496,0.175423,0.090047,-0.264996,0.131081,-0.938899
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48027,0.144812,-0.206902,0.387329,-0.437603,0.050504,0.616085,0.354959,0.615743,0.199554,0.350018,...,0.056309,0.418520,-0.243358,0.067578,-0.259211,-0.125076,0.046033,-0.104588,0.535638,-0.250660
48028,-0.247568,0.034001,0.403492,-0.373971,-0.078511,0.362194,-0.348690,0.319429,0.159658,0.068191,...,-0.115766,0.427706,-0.349669,0.134592,0.217702,0.257980,0.151721,-0.488177,-0.031998,-0.287062
48029,-0.129721,-0.259378,0.618387,-0.082403,-0.767531,0.469959,-0.079017,0.475196,0.058982,-0.041019,...,-0.368901,0.584279,-0.539207,0.455483,-0.068754,0.196912,0.300624,-0.041595,0.361152,-0.228904
48030,0.230608,-0.291521,0.692082,-0.415089,-0.281493,0.432509,0.277594,0.677817,0.109448,0.317037,...,-0.334936,0.189219,-0.407189,0.137793,-0.125190,0.116713,-0.139185,-0.297847,0.170795,-0.421031


In [21]:
df_max_scaled['ID'] = embeddings['ID']
embeddings = df_max_scaled 

In [22]:
embeddings_test = embeddings.iloc[:14410,]
embeddings_test

,emb_0,emb_1,emb_2,emb_3,emb_4,emb_5,emb_6,emb_7,emb_8,emb_9,...,emb_1271,emb_1272,emb_1273,emb_1274,emb_1275,emb_1276,emb_1277,emb_1278,emb_1279,ID
0,0.163292,-0.214504,0.772762,-0.100303,-0.188323,0.325881,-0.000539,0.468595,0.308145,0.134281,...,0.457562,-0.353930,0.465998,-0.168417,0.260290,0.162296,-0.180027,0.441839,0.294106,test_0
1,0.009500,-0.225702,0.639525,-0.101877,-0.323824,0.479880,-0.393520,0.440001,0.407549,0.502149,...,0.123206,-0.329576,0.329371,0.003080,0.292193,-0.027474,-0.041855,0.344309,-0.420055,test_1
2,-0.177092,-0.234253,0.614128,-0.141611,-0.516879,0.465106,-0.176217,0.596966,0.012103,-0.108849,...,0.411306,-0.405391,0.477918,0.036448,0.235378,0.267199,-0.303903,0.496732,-0.294776,test_10
3,0.441019,-0.185077,0.756106,-0.386226,-0.148177,0.322777,0.026444,0.709156,-0.241100,0.518319,...,0.374362,-0.271777,0.189914,0.035025,-0.257808,0.029366,-0.279828,0.100594,-0.496086,test_100
4,-0.019138,-0.303671,0.578037,-0.347874,-0.345237,0.564734,-0.397410,0.666537,0.122159,-0.084848,...,0.354293,-0.386782,0.504587,0.106496,0.175423,0.090047,-0.264996,0.131081,-0.938899,test_1000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14405,-0.045759,-0.157574,0.450432,-0.222323,-0.355817,0.727717,-0.004630,0.465954,0.032782,0.307331,...,0.256800,-0.698933,0.159612,-0.271765,0.132447,0.177108,-0.127528,0.363608,0.073110,test_9995
14406,0.304801,-0.454637,0.709497,-0.428932,-0.418213,0.524086,-0.053738,0.638880,-0.378901,0.211980,...,0.418548,-0.609835,0.361062,0.115266,0.039395,-0.053983,-0.242532,0.091590,0.290155,test_9996
14407,-0.172103,0.015729,0.506233,-0.071067,-0.614866,0.650367,-0.430669,0.669392,-0.038041,0.315921,...,0.353905,-0.389881,0.225133,-0.151836,0.130654,0.063417,0.063525,0.149866,-0.108796,test_9997
14408,0.160073,-0.224252,0.468935,-0.025844,-0.498145,0.626574,-0.310079,0.504651,0.178102,0.183846,...,0.540655,-0.493611,0.381259,-0.080976,0.160376,-0.322413,-0.069231,0.186773,-0.393347,test_9998


In [23]:
embeddings_train = embeddings.iloc[14410:,].reset_index(drop = True)
embeddings_train

,emb_0,emb_1,emb_2,emb_3,emb_4,emb_5,emb_6,emb_7,emb_8,emb_9,...,emb_1271,emb_1272,emb_1273,emb_1274,emb_1275,emb_1276,emb_1277,emb_1278,emb_1279,ID
0,0.014645,-0.147870,0.476448,0.100457,-0.240648,0.486970,-0.089564,0.423598,-0.091858,-0.077804,...,0.442393,-0.351364,0.228832,-0.026886,0.353022,0.290211,-0.209606,0.415990,-0.273816,train_0
1,-0.060972,-0.064082,0.447789,-0.092881,-0.497990,0.468571,-0.265350,0.493563,-0.139367,0.094702,...,0.381895,-0.428084,0.349921,-0.118833,0.124103,-0.368222,0.179080,-0.271503,-0.384494,train_1
2,0.317332,-0.146892,0.558799,-0.283120,-0.273099,0.434951,-0.284497,0.547655,-0.069823,0.447782,...,0.216051,-0.157245,0.172128,0.017433,0.216000,-0.013789,-0.197020,0.501589,0.213663,train_10
3,-0.156965,-0.074715,0.519195,0.025081,-0.413589,0.555275,-0.366223,0.611189,0.011607,0.065204,...,0.300367,-0.635760,0.312799,0.012043,0.164169,-0.309875,0.158825,0.066475,-0.245937,train_100
4,0.246723,-0.146822,0.516904,-0.016756,-0.442620,0.496439,0.194371,0.566249,0.354867,0.472162,...,0.297603,-0.507172,0.359391,-0.205108,0.359141,-0.172220,-0.068285,0.261426,-0.204564,train_1000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33617,0.144812,-0.206902,0.387329,-0.437603,0.050504,0.616085,0.354959,0.615743,0.199554,0.350018,...,0.418520,-0.243358,0.067578,-0.259211,-0.125076,0.046033,-0.104588,0.535638,-0.250660,train_9995
33618,-0.247568,0.034001,0.403492,-0.373971,-0.078511,0.362194,-0.348690,0.319429,0.159658,0.068191,...,0.427706,-0.349669,0.134592,0.217702,0.257980,0.151721,-0.488177,-0.031998,-0.287062,train_9996
33619,-0.129721,-0.259378,0.618387,-0.082403,-0.767531,0.469959,-0.079017,0.475196,0.058982,-0.041019,...,0.584279,-0.539207,0.455483,-0.068754,0.196912,0.300624,-0.041595,0.361152,-0.228904,train_9997
33620,0.230608,-0.291521,0.692082,-0.415089,-0.281493,0.432509,0.277594,0.677817,0.109448,0.317037,...,0.189219,-0.407189,0.137793,-0.125190,0.116713,-0.139185,-0.297847,0.170795,-0.421031,train_9998


In [24]:
embeddings_train['sort'] = embeddings_train['ID'].str.extract('(\d+)', expand=False).astype(int)

embeddings_train = embeddings_train.sort_values('sort')

embeddings_train.drop('sort', axis=1, inplace = True)

embeddings_train

,emb_0,emb_1,emb_2,emb_3,emb_4,emb_5,emb_6,emb_7,emb_8,emb_9,...,emb_1271,emb_1272,emb_1273,emb_1274,emb_1275,emb_1276,emb_1277,emb_1278,emb_1279,ID
0,0.014645,-0.147870,0.476448,0.100457,-0.240648,0.486970,-0.089564,0.423598,-0.091858,-0.077804,...,0.442393,-0.351364,0.228832,-0.026886,0.353022,0.290211,-0.209606,0.415990,-0.273816,train_0
1,-0.060972,-0.064082,0.447789,-0.092881,-0.497990,0.468571,-0.265350,0.493563,-0.139367,0.094702,...,0.381895,-0.428084,0.349921,-0.118833,0.124103,-0.368222,0.179080,-0.271503,-0.384494,train_1
11112,0.388171,-0.332602,0.686292,-0.686215,-0.403664,0.226897,-0.291076,0.465988,0.514434,0.560383,...,0.073539,-0.293384,0.318075,0.223095,0.247016,0.097312,-0.163520,0.492443,0.142338,train_2
22223,0.110690,-0.337533,0.575088,-0.053165,-0.310254,0.243119,-0.118179,0.248213,0.082652,0.203539,...,0.208338,-0.340593,0.479740,0.250047,0.468662,0.369809,-0.234239,0.522625,0.039038,train_3
26956,-0.206384,-0.202387,0.800344,-0.085418,-0.312244,0.490253,0.229997,0.451635,0.082014,0.246026,...,0.442282,-0.503629,0.485226,-0.074759,-0.225607,-0.446840,-0.091173,0.048944,-0.306805,train_4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26244,0.293673,-0.415119,0.536873,0.021775,-0.117309,0.459858,-0.140650,0.480937,0.001267,0.358217,...,0.625171,0.058947,0.549459,0.105616,0.398511,-0.309805,-0.458766,0.285866,-0.100197,train_33617
26245,0.262948,-0.359393,0.584479,-0.283629,-0.163383,0.233655,0.010876,0.523676,0.063749,0.312774,...,0.352142,-0.268261,0.613508,0.054264,0.086447,-0.079620,-0.032677,0.519736,-0.127810,train_33618
26246,0.382920,-0.417154,0.394925,-0.509775,-0.550469,0.626975,-0.007609,0.446708,0.341299,-0.176618,...,0.388227,-0.112671,0.260953,-0.320057,0.369863,0.296034,-0.419130,0.492045,-0.332331,train_33619
26248,0.211578,-0.136949,0.638623,0.079982,-0.521569,0.490353,-0.246674,0.585639,0.071479,0.126197,...,0.412035,-0.562120,0.202634,0.368022,0.056083,-0.272934,-0.181818,0.296293,-0.136584,train_33620


In [25]:
embeddings_train.reset_index(drop = True, inplace = True)
embeddings_train

,emb_0,emb_1,emb_2,emb_3,emb_4,emb_5,emb_6,emb_7,emb_8,emb_9,...,emb_1271,emb_1272,emb_1273,emb_1274,emb_1275,emb_1276,emb_1277,emb_1278,emb_1279,ID
0,0.014645,-0.147870,0.476448,0.100457,-0.240648,0.486970,-0.089564,0.423598,-0.091858,-0.077804,...,0.442393,-0.351364,0.228832,-0.026886,0.353022,0.290211,-0.209606,0.415990,-0.273816,train_0
1,-0.060972,-0.064082,0.447789,-0.092881,-0.497990,0.468571,-0.265350,0.493563,-0.139367,0.094702,...,0.381895,-0.428084,0.349921,-0.118833,0.124103,-0.368222,0.179080,-0.271503,-0.384494,train_1
2,0.388171,-0.332602,0.686292,-0.686215,-0.403664,0.226897,-0.291076,0.465988,0.514434,0.560383,...,0.073539,-0.293384,0.318075,0.223095,0.247016,0.097312,-0.163520,0.492443,0.142338,train_2
3,0.110690,-0.337533,0.575088,-0.053165,-0.310254,0.243119,-0.118179,0.248213,0.082652,0.203539,...,0.208338,-0.340593,0.479740,0.250047,0.468662,0.369809,-0.234239,0.522625,0.039038,train_3
4,-0.206384,-0.202387,0.800344,-0.085418,-0.312244,0.490253,0.229997,0.451635,0.082014,0.246026,...,0.442282,-0.503629,0.485226,-0.074759,-0.225607,-0.446840,-0.091173,0.048944,-0.306805,train_4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33617,0.293673,-0.415119,0.536873,0.021775,-0.117309,0.459858,-0.140650,0.480937,0.001267,0.358217,...,0.625171,0.058947,0.549459,0.105616,0.398511,-0.309805,-0.458766,0.285866,-0.100197,train_33617
33618,0.262948,-0.359393,0.584479,-0.283629,-0.163383,0.233655,0.010876,0.523676,0.063749,0.312774,...,0.352142,-0.268261,0.613508,0.054264,0.086447,-0.079620,-0.032677,0.519736,-0.127810,train_33618
33619,0.382920,-0.417154,0.394925,-0.509775,-0.550469,0.626975,-0.007609,0.446708,0.341299,-0.176618,...,0.388227,-0.112671,0.260953,-0.320057,0.369863,0.296034,-0.419130,0.492045,-0.332331,train_33619
33620,0.211578,-0.136949,0.638623,0.079982,-0.521569,0.490353,-0.246674,0.585639,0.071479,0.126197,...,0.412035,-0.562120,0.202634,0.368022,0.056083,-0.272934,-0.181818,0.296293,-0.136584,train_33620


In [26]:
embeddings_test['sort'] = embeddings_test['ID'].str.extract('(\d+)', expand=False).astype(int)

embeddings_test = embeddings_test.sort_values('sort')

embeddings_test.drop('sort', axis=1, inplace = True)

embeddings_test

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,emb_0,emb_1,emb_2,emb_3,emb_4,emb_5,emb_6,emb_7,emb_8,emb_9,...,emb_1271,emb_1272,emb_1273,emb_1274,emb_1275,emb_1276,emb_1277,emb_1278,emb_1279,ID
0,0.163292,-0.214504,0.772762,-0.100303,-0.188323,0.325881,-0.000539,0.468595,0.308145,0.134281,...,0.457562,-0.353930,0.465998,-0.168417,0.260290,0.162296,-0.180027,0.441839,0.294106,test_0
1,0.009500,-0.225702,0.639525,-0.101877,-0.323824,0.479880,-0.393520,0.440001,0.407549,0.502149,...,0.123206,-0.329576,0.329371,0.003080,0.292193,-0.027474,-0.041855,0.344309,-0.420055,test_1
5522,0.228945,-0.168050,0.683938,-0.142356,-0.632106,0.447705,-0.041977,0.300667,-0.292377,0.427358,...,0.262730,-0.337760,0.135428,-0.103995,0.205991,-0.099295,-0.005142,-0.008736,-0.157766,test_2
6633,0.636058,-0.200055,0.700809,-0.184726,-0.127384,0.330389,-0.215493,0.590628,-0.260808,0.410265,...,0.382872,-0.171567,0.224555,-0.199805,-0.309741,0.047881,-0.240072,-0.010063,-0.340989,test_3
7744,0.546962,-0.342608,0.693249,0.082553,-0.233084,0.538250,-0.139242,0.650262,-0.251738,0.151553,...,0.415507,-0.209327,0.221639,0.061116,0.278070,0.105232,0.094663,0.289922,0.330935,test_4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4898,0.612782,-0.276367,0.512292,-0.269181,-0.463089,0.507511,0.227465,0.330812,0.373474,0.231820,...,0.509862,0.107128,0.546657,-0.091123,0.224462,0.278414,0.021606,0.299201,0.295165,test_14405
4899,0.080309,-0.197527,0.692253,-0.246466,-0.334418,0.406325,-0.031575,0.678697,0.375195,-0.016483,...,0.153664,-0.576759,0.168888,0.121167,0.019305,-0.111658,-0.175915,0.243330,0.124931,test_14406
4900,-0.146013,-0.230650,0.545786,-0.431125,-0.294524,0.656581,-0.116434,0.399291,0.093945,-0.254333,...,0.101865,-0.417124,0.334227,-0.036115,0.323025,0.010913,-0.367958,0.260395,-0.317786,test_14407
4901,0.272690,-0.354203,0.674278,-0.329704,-0.243256,0.532942,-0.254644,0.426843,0.210993,0.641944,...,0.500592,-0.141038,0.309110,0.010035,0.249310,-0.067415,-0.194301,0.601052,-0.198212,test_14408


In [27]:
embeddings_test.reset_index(drop = True, inplace = True)
embeddings_test

,emb_0,emb_1,emb_2,emb_3,emb_4,emb_5,emb_6,emb_7,emb_8,emb_9,...,emb_1271,emb_1272,emb_1273,emb_1274,emb_1275,emb_1276,emb_1277,emb_1278,emb_1279,ID
0,0.163292,-0.214504,0.772762,-0.100303,-0.188323,0.325881,-0.000539,0.468595,0.308145,0.134281,...,0.457562,-0.353930,0.465998,-0.168417,0.260290,0.162296,-0.180027,0.441839,0.294106,test_0
1,0.009500,-0.225702,0.639525,-0.101877,-0.323824,0.479880,-0.393520,0.440001,0.407549,0.502149,...,0.123206,-0.329576,0.329371,0.003080,0.292193,-0.027474,-0.041855,0.344309,-0.420055,test_1
2,0.228945,-0.168050,0.683938,-0.142356,-0.632106,0.447705,-0.041977,0.300667,-0.292377,0.427358,...,0.262730,-0.337760,0.135428,-0.103995,0.205991,-0.099295,-0.005142,-0.008736,-0.157766,test_2
3,0.636058,-0.200055,0.700809,-0.184726,-0.127384,0.330389,-0.215493,0.590628,-0.260808,0.410265,...,0.382872,-0.171567,0.224555,-0.199805,-0.309741,0.047881,-0.240072,-0.010063,-0.340989,test_3
4,0.546962,-0.342608,0.693249,0.082553,-0.233084,0.538250,-0.139242,0.650262,-0.251738,0.151553,...,0.415507,-0.209327,0.221639,0.061116,0.278070,0.105232,0.094663,0.289922,0.330935,test_4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14405,0.612782,-0.276367,0.512292,-0.269181,-0.463089,0.507511,0.227465,0.330812,0.373474,0.231820,...,0.509862,0.107128,0.546657,-0.091123,0.224462,0.278414,0.021606,0.299201,0.295165,test_14405
14406,0.080309,-0.197527,0.692253,-0.246466,-0.334418,0.406325,-0.031575,0.678697,0.375195,-0.016483,...,0.153664,-0.576759,0.168888,0.121167,0.019305,-0.111658,-0.175915,0.243330,0.124931,test_14406
14407,-0.146013,-0.230650,0.545786,-0.431125,-0.294524,0.656581,-0.116434,0.399291,0.093945,-0.254333,...,0.101865,-0.417124,0.334227,-0.036115,0.323025,0.010913,-0.367958,0.260395,-0.317786,test_14407
14408,0.272690,-0.354203,0.674278,-0.329704,-0.243256,0.532942,-0.254644,0.426843,0.210993,0.641944,...,0.500592,-0.141038,0.309110,0.010035,0.249310,-0.067415,-0.194301,0.601052,-0.198212,test_14408


In [28]:
train = pd.concat([df, embeddings_train.reindex(df.index)], axis=1)
train

,ID,protein_name,sequence,target,emb_0,emb_1,emb_2,emb_3,emb_4,emb_5,...,emb_1271,emb_1272,emb_1273,emb_1274,emb_1275,emb_1276,emb_1277,emb_1278,emb_1279,ID
0,train_0,sp|Q9J523|V212_FOWPN,MPQNKVLSFPLPEGTLLEDITKNKWILGKQLGSGGFGLVYQVSCKS...,2,0.014645,-0.147870,0.476448,0.100457,-0.240648,0.486970,...,0.442393,-0.351364,0.228832,-0.026886,0.353022,0.290211,-0.209606,0.415990,-0.273816,train_0
1,train_1,sp|Q14990|ODFP1_HUMAN,MAALSCLLDSVRRDIKKVDRELRQLRCIDEFSTRCLCDLYMHPYCC...,2,-0.060972,-0.064082,0.447789,-0.092881,-0.497990,0.468571,...,0.381895,-0.428084,0.349921,-0.118833,0.124103,-0.368222,0.179080,-0.271503,-0.384494,train_1
2,train_2,sp|Q9C994|DTX14_ARATH,MDSAEKGLLVVSDREEVNKKDGFLRETKKLSYIAGPMIAVNSSMYV...,2,0.388171,-0.332602,0.686292,-0.686215,-0.403664,0.226897,...,0.073539,-0.293384,0.318075,0.223095,0.247016,0.097312,-0.163520,0.492443,0.142338,train_2
3,train_3,sp|Q58425|CARA_METJA,MEAVLILEDGTILKGKGFGAEKEVFGELVFTTVMTGYVEVLTDPSY...,1,0.110690,-0.337533,0.575088,-0.053165,-0.310254,0.243119,...,0.208338,-0.340593,0.479740,0.250047,0.468662,0.369809,-0.234239,0.522625,0.039038,train_3
4,train_4,sp|P78814|SRP2_SCHPO,MSETRLFVGRIPPQATREDMMDFFKGYGQILDCKLMNGFGFVEVED...,2,-0.206384,-0.202387,0.800344,-0.085418,-0.312244,0.490253,...,0.442282,-0.503629,0.485226,-0.074759,-0.225607,-0.446840,-0.091173,0.048944,-0.306805,train_4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33617,train_33617,sp|Q6ED58|CYB_STABA,MSIRKTNPFISMANSYVMDAPEPSNMSYFWNFGSLLGVCLVMQLCT...,2,0.293673,-0.415119,0.536873,0.021775,-0.117309,0.459858,...,0.625171,0.058947,0.549459,0.105616,0.398511,-0.309805,-0.458766,0.285866,-0.100197,train_33617
33618,train_33618,sp|Q9LRP2|PT317_ARATH,MAKMINKTLVLTYIYLLIYIILSSGVILYNKWVLSPKYFNFPLPIT...,1,0.262948,-0.359393,0.584479,-0.283629,-0.163383,0.233655,...,0.352142,-0.268261,0.613508,0.054264,0.086447,-0.079620,-0.032677,0.519736,-0.127810,train_33618
33619,train_33619,sp|B0R5P8|PANB_HALS3,MPTVNSVRRAGNTDEETDPVTMLTAYDAPTASVVDDAGVDMILVGD...,1,0.382920,-0.417154,0.394925,-0.509775,-0.550469,0.626975,...,0.388227,-0.112671,0.260953,-0.320057,0.369863,0.296034,-0.419130,0.492045,-0.332331,train_33619
33620,train_33620,sp|D4N3P3|CAPSD_HCYV5,MPFSRRRRVVRRRKPVRRLRRRRRRFFKRAGRGSFRVKLTRFVSIT...,0,0.211578,-0.136949,0.638623,0.079982,-0.521569,0.490353,...,0.412035,-0.562120,0.202634,0.368022,0.056083,-0.272934,-0.181818,0.296293,-0.136584,train_33620


In [29]:
train.drop('ID', axis=1, inplace = True)
train.drop('sequence', axis=1, inplace = True)
train.drop('protein_name', axis=1, inplace = True)

In [30]:
train

,target,emb_0,emb_1,emb_2,emb_3,emb_4,emb_5,emb_6,emb_7,emb_8,...,emb_1270,emb_1271,emb_1272,emb_1273,emb_1274,emb_1275,emb_1276,emb_1277,emb_1278,emb_1279
0,2,0.014645,-0.147870,0.476448,0.100457,-0.240648,0.486970,-0.089564,0.423598,-0.091858,...,0.006958,0.442393,-0.351364,0.228832,-0.026886,0.353022,0.290211,-0.209606,0.415990,-0.273816
1,2,-0.060972,-0.064082,0.447789,-0.092881,-0.497990,0.468571,-0.265350,0.493563,-0.139367,...,-0.100938,0.381895,-0.428084,0.349921,-0.118833,0.124103,-0.368222,0.179080,-0.271503,-0.384494
2,2,0.388171,-0.332602,0.686292,-0.686215,-0.403664,0.226897,-0.291076,0.465988,0.514434,...,-0.386167,0.073539,-0.293384,0.318075,0.223095,0.247016,0.097312,-0.163520,0.492443,0.142338
3,1,0.110690,-0.337533,0.575088,-0.053165,-0.310254,0.243119,-0.118179,0.248213,0.082652,...,-0.360609,0.208338,-0.340593,0.479740,0.250047,0.468662,0.369809,-0.234239,0.522625,0.039038
4,2,-0.206384,-0.202387,0.800344,-0.085418,-0.312244,0.490253,0.229997,0.451635,0.082014,...,-0.136507,0.442282,-0.503629,0.485226,-0.074759,-0.225607,-0.446840,-0.091173,0.048944,-0.306805
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33617,2,0.293673,-0.415119,0.536873,0.021775,-0.117309,0.459858,-0.140650,0.480937,0.001267,...,-0.367929,0.625171,0.058947,0.549459,0.105616,0.398511,-0.309805,-0.458766,0.285866,-0.100197
33618,1,0.262948,-0.359393,0.584479,-0.283629,-0.163383,0.233655,0.010876,0.523676,0.063749,...,-0.451873,0.352142,-0.268261,0.613508,0.054264,0.086447,-0.079620,-0.032677,0.519736,-0.127810
33619,1,0.382920,-0.417154,0.394925,-0.509775,-0.550469,0.626975,-0.007609,0.446708,0.341299,...,0.138812,0.388227,-0.112671,0.260953,-0.320057,0.369863,0.296034,-0.419130,0.492045,-0.332331
33620,0,0.211578,-0.136949,0.638623,0.079982,-0.521569,0.490353,-0.246674,0.585639,0.071479,...,-0.421462,0.412035,-0.562120,0.202634,0.368022,0.056083,-0.272934,-0.181818,0.296293,-0.136584


In [31]:
test = pd.concat([df_test, embeddings_test.reindex(df_test.index)], axis=1)
test

,ID,protein_name,sequence,emb_0,emb_1,emb_2,emb_3,emb_4,emb_5,emb_6,...,emb_1271,emb_1272,emb_1273,emb_1274,emb_1275,emb_1276,emb_1277,emb_1278,emb_1279,ID
0,test_0,sp|O02056|RL4_CAEEL,MAARPLVTVYDEKYEATQSQIRLPAVFRTPIRPDLVSFIADQVRRN...,0.163292,-0.214504,0.772762,-0.100303,-0.188323,0.325881,-0.000539,...,0.457562,-0.353930,0.465998,-0.168417,0.260290,0.162296,-0.180027,0.441839,0.294106,test_0
1,test_1,sp|G4RJZ4|CS3HD_THETK,MSCCAYFRGRDCLQTYEDHITQALEACERLRPYYGRWMEKAFGTAD...,0.009500,-0.225702,0.639525,-0.101877,-0.323824,0.479880,-0.393520,...,0.123206,-0.329576,0.329371,0.003080,0.292193,-0.027474,-0.041855,0.344309,-0.420055,test_1
2,test_2,sp|P03109|VL2_BPV1,MSARKRVKRASAYDLYRTCKQAGTCPPDVIPKVEGDTIADKILKFG...,0.228945,-0.168050,0.683938,-0.142356,-0.632106,0.447705,-0.041977,...,0.262730,-0.337760,0.135428,-0.103995,0.205991,-0.099295,-0.005142,-0.008736,-0.157766,test_2
3,test_3,sp|Q61473|SOX17_MOUSE,MSSPDAGYASDDQSQPRSAQPAVMAGLGPCPWAESLSPLGDVKVKG...,0.636058,-0.200055,0.700809,-0.184726,-0.127384,0.330389,-0.215493,...,0.382872,-0.171567,0.224555,-0.199805,-0.309741,0.047881,-0.240072,-0.010063,-0.340989,test_3
4,test_4,sp|P51641|CNTFR_CHICK,MANPVPSACCVVLAAVVVVYAQRHSQQDSHIQYERVGADVTMKCGS...,0.546962,-0.342608,0.693249,0.082553,-0.233084,0.538250,-0.139242,...,0.415507,-0.209327,0.221639,0.061116,0.278070,0.105232,0.094663,0.289922,0.330935,test_4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14405,test_14405,sp|Q6ZHE5|DCYD1_ORYSJ,MARGAHQAPGGFWTVAAAPTRCSLPHSLPIPLHAAAAAAAWMAGVS...,0.612782,-0.276367,0.512292,-0.269181,-0.463089,0.507511,0.227465,...,0.509862,0.107128,0.546657,-0.091123,0.224462,0.278414,0.021606,0.299201,0.295165,test_14405
14406,test_14406,sp|P53164|NPY1_YEAST,MSTAVTFFGQHVLNRVSFLRCSKEFIKKSLNHDSTVFIPFIEGEAL...,0.080309,-0.197527,0.692253,-0.246466,-0.334418,0.406325,-0.031575,...,0.153664,-0.576759,0.168888,0.121167,0.019305,-0.111658,-0.175915,0.243330,0.124931,test_14406
14407,test_14407,sp|Q9YE02|SPEE_AERPE,MAGGSEKSVFLKWSWFLEWLTPDRATLKHIEDVIFQGRSRFQEIAV...,-0.146013,-0.230650,0.545786,-0.431125,-0.294524,0.656581,-0.116434,...,0.101865,-0.417124,0.334227,-0.036115,0.323025,0.010913,-0.367958,0.260395,-0.317786,test_14407
14408,test_14408,sp|Q8VXY7|ENT1_ARATH,MTPISQRISQKSPKLRETKQIQMTTTDKSAGIVTDSEAGPETSLLL...,0.272690,-0.354203,0.674278,-0.329704,-0.243256,0.532942,-0.254644,...,0.500592,-0.141038,0.309110,0.010035,0.249310,-0.067415,-0.194301,0.601052,-0.198212,test_14408


In [32]:
test.drop('ID', axis=1, inplace = True)
test.drop('sequence', axis=1, inplace = True)
test.drop('protein_name', axis=1, inplace = True)
test

,emb_0,emb_1,emb_2,emb_3,emb_4,emb_5,emb_6,emb_7,emb_8,emb_9,...,emb_1270,emb_1271,emb_1272,emb_1273,emb_1274,emb_1275,emb_1276,emb_1277,emb_1278,emb_1279
0,0.163292,-0.214504,0.772762,-0.100303,-0.188323,0.325881,-0.000539,0.468595,0.308145,0.134281,...,-0.321614,0.457562,-0.353930,0.465998,-0.168417,0.260290,0.162296,-0.180027,0.441839,0.294106
1,0.009500,-0.225702,0.639525,-0.101877,-0.323824,0.479880,-0.393520,0.440001,0.407549,0.502149,...,-0.215763,0.123206,-0.329576,0.329371,0.003080,0.292193,-0.027474,-0.041855,0.344309,-0.420055
2,0.228945,-0.168050,0.683938,-0.142356,-0.632106,0.447705,-0.041977,0.300667,-0.292377,0.427358,...,-0.164225,0.262730,-0.337760,0.135428,-0.103995,0.205991,-0.099295,-0.005142,-0.008736,-0.157766
3,0.636058,-0.200055,0.700809,-0.184726,-0.127384,0.330389,-0.215493,0.590628,-0.260808,0.410265,...,-0.457233,0.382872,-0.171567,0.224555,-0.199805,-0.309741,0.047881,-0.240072,-0.010063,-0.340989
4,0.546962,-0.342608,0.693249,0.082553,-0.233084,0.538250,-0.139242,0.650262,-0.251738,0.151553,...,-0.456776,0.415507,-0.209327,0.221639,0.061116,0.278070,0.105232,0.094663,0.289922,0.330935
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14405,0.612782,-0.276367,0.512292,-0.269181,-0.463089,0.507511,0.227465,0.330812,0.373474,0.231820,...,-0.522819,0.509862,0.107128,0.546657,-0.091123,0.224462,0.278414,0.021606,0.299201,0.295165
14406,0.080309,-0.197527,0.692253,-0.246466,-0.334418,0.406325,-0.031575,0.678697,0.375195,-0.016483,...,-0.113387,0.153664,-0.576759,0.168888,0.121167,0.019305,-0.111658,-0.175915,0.243330,0.124931
14407,-0.146013,-0.230650,0.545786,-0.431125,-0.294524,0.656581,-0.116434,0.399291,0.093945,-0.254333,...,-0.104526,0.101865,-0.417124,0.334227,-0.036115,0.323025,0.010913,-0.367958,0.260395,-0.317786
14408,0.272690,-0.354203,0.674278,-0.329704,-0.243256,0.532942,-0.254644,0.426843,0.210993,0.641944,...,-0.550734,0.500592,-0.141038,0.309110,0.010035,0.249310,-0.067415,-0.194301,0.601052,-0.198212


## PCA

In [91]:
from sklearn.preprocessing import StandardScaler # to standardize the features
from sklearn.decomposition import PCA # to apply PCA
import seaborn as sns # to plot the heat maps

In [58]:
data_to_compress = pd.concat([train,test])
data_to_compress

,target,emb_0,emb_1,emb_2,emb_3,emb_4,emb_5,emb_6,emb_7,emb_8,...,emb_1270,emb_1271,emb_1272,emb_1273,emb_1274,emb_1275,emb_1276,emb_1277,emb_1278,emb_1279
0,2.0,0.081642,-1.496138,6.191256,0.679270,-1.894954,4.502281,-0.514480,3.705019,-0.492262,...,0.039820,3.243101,-2.772209,2.121812,-0.186634,2.419172,1.625475,-1.370067,3.178513,-1.555542
1,2.0,-0.339906,-0.648377,5.818838,-0.628044,-3.921365,4.332178,-1.524251,4.316974,-0.746856,...,-0.577698,2.799602,-3.377520,3.244596,-0.824896,0.850447,-2.062417,1.170538,-2.074513,-2.184298
2,2.0,2.163980,-3.365240,8.918085,-4.640063,-3.178599,2.097780,-1.672030,4.075786,2.756809,...,-2.210143,0.539102,-2.314760,2.949310,1.548649,1.692737,0.545045,-1.068834,3.762681,0.808617
3,1.0,0.617074,-3.415126,7.473043,-0.359494,-2.443056,2.247761,-0.678855,2.171010,0.442927,...,-2.063864,1.527290,-2.687233,4.448330,1.735738,3.211624,2.071304,-1.531078,3.993294,0.221772
4,2.0,-1.150551,-2.047733,10.400146,-0.577579,-2.458724,4.532634,1.321174,3.950249,0.439508,...,-0.781267,3.242286,-3.973563,4.499197,-0.518952,-1.546026,-2.502755,-0.595940,0.373970,-1.742952
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14405,NaN,3.416143,-2.796250,6.657033,-1.820151,-3.646534,4.692196,1.306629,2.893468,2.001418,...,-2.992237,3.737702,0.845226,5.068807,-0.632543,1.538181,1.559400,0.141228,2.286150,1.676825
14406,NaN,0.447707,-1.998561,8.995557,-1.666557,-2.633336,3.756681,-0.181378,5.936257,2.010641,...,-0.648948,1.126485,-4.550549,1.565988,0.841098,0.132292,-0.625396,-1.149852,1.859243,0.709729
14407,NaN,-0.813993,-2.333690,7.092276,-2.915187,-2.319193,6.070423,-0.668832,3.492422,0.503443,...,-0.598230,0.746757,-3.291050,3.099078,-0.250695,2.213611,0.061124,-2.405120,1.989640,-1.805332
14408,NaN,1.520194,-3.583797,8.761969,-2.229401,-1.915491,4.927318,-1.462749,3.733409,1.130696,...,-3.152003,3.669749,-1.112769,2.866182,0.069662,1.708462,-0.377591,-1.270029,4.592544,-1.126035


In [62]:
pca = PCA(n_components = 100)
pca.fit(data_to_compress.drop('target',axis = 1))
data_pca = pca.transform(data_to_compress.drop('target',axis = 1))
data_pca = pd.DataFrame(data_pca,columns=[ 'Pc'+str(i) for i in range(100)])
data_pca.head()

,Pc0,Pc1,Pc2,Pc3,Pc4,Pc5,Pc6,Pc7,Pc8,Pc9,...,Pc90,Pc91,Pc92,Pc93,Pc94,Pc95,Pc96,Pc97,Pc98,Pc99
0,6.080757,7.954875,-2.802692,2.653759,4.011957,-1.882538,0.594221,-9.733253,2.039697,4.315181,...,3.461804,-1.447385,1.617427,-1.351892,1.422038,-0.240246,-0.785429,0.665951,-0.759634,-4.678932
1,30.036447,-21.873893,3.636547,-1.850623,1.153098,7.418425,-3.877800,2.782597,0.858048,-6.112473,...,-0.450306,0.232597,-0.208322,0.866749,-1.054738,0.435261,0.155073,0.231425,1.485202,1.096075
2,-12.107453,16.221957,22.471516,0.424821,5.054502,-11.044553,-2.081414,0.694179,-6.138139,-1.522114,...,-2.546936,-0.368128,2.776458,1.519931,1.090265,0.409069,1.210661,-0.946403,1.408324,-0.607774
3,1.346085,23.551188,-7.554624,-2.560278,6.970243,3.910359,3.918008,-0.263672,-4.631270,-5.018888,...,7.480651,0.404432,-1.352761,5.270492,6.401115,0.102558,-3.367166,2.121251,2.746955,4.293790
4,1.880267,-5.478529,-8.641508,-13.674802,9.698266,3.338851,5.228863,0.741220,-3.902216,-0.839658,...,-1.592892,-1.602891,2.937229,4.845227,-1.596047,3.014279,-0.140397,-0.919787,1.552691,-0.941675


In [63]:
train_pca = data_pca.iloc[:33622,].reset_index(drop = True)
train_pca

,Pc0,Pc1,Pc2,Pc3,Pc4,Pc5,Pc6,Pc7,Pc8,Pc9,...,Pc90,Pc91,Pc92,Pc93,Pc94,Pc95,Pc96,Pc97,Pc98,Pc99
0,6.080757,7.954875,-2.802692,2.653759,4.011957,-1.882538,0.594221,-9.733253,2.039697,4.315181,...,3.461804,-1.447385,1.617427,-1.351892,1.422038,-0.240246,-0.785429,0.665951,-0.759634,-4.678932
1,30.036447,-21.873893,3.636547,-1.850623,1.153098,7.418425,-3.877800,2.782597,0.858048,-6.112473,...,-0.450306,0.232597,-0.208322,0.866749,-1.054738,0.435261,0.155073,0.231425,1.485202,1.096075
2,-12.107453,16.221957,22.471516,0.424821,5.054502,-11.044553,-2.081414,0.694179,-6.138139,-1.522114,...,-2.546936,-0.368128,2.776458,1.519931,1.090265,0.409069,1.210661,-0.946403,1.408324,-0.607774
3,1.346085,23.551188,-7.554624,-2.560278,6.970243,3.910359,3.918008,-0.263672,-4.631270,-5.018888,...,7.480651,0.404432,-1.352761,5.270492,6.401115,0.102558,-3.367166,2.121251,2.746955,4.293790
4,1.880267,-5.478529,-8.641508,-13.674802,9.698266,3.338851,5.228863,0.741220,-3.902216,-0.839658,...,-1.592892,-1.602891,2.937229,4.845227,-1.596047,3.014279,-0.140397,-0.919787,1.552691,-0.941675
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33617,17.460651,23.549214,22.178911,-4.954198,8.310337,-6.282133,10.369579,7.570798,0.656124,1.357701,...,0.130161,-4.213453,-5.313589,-6.332882,4.621230,4.585040,0.232472,4.850231,5.790078,-3.050809
33618,-30.861525,11.351626,20.171907,-3.803731,-6.453376,-15.332609,1.663190,1.987642,-1.803880,-4.017764,...,-3.646183,-0.902648,3.671061,2.179107,0.026893,-2.669256,-0.447677,-1.557754,1.032013,1.159668
33619,-0.303412,24.541428,-6.630254,-14.424322,1.281926,9.405782,-3.889374,5.640531,0.160012,-4.657691,...,-3.822818,0.128933,-7.730116,0.851782,-0.364746,-4.354710,-1.967472,1.234557,-1.556982,-6.643284
33620,16.179192,-1.277407,-0.925478,-1.143221,-5.675464,6.141842,16.658610,3.870176,-6.176522,2.710918,...,-4.184337,-1.655715,3.008332,-5.446652,-0.947447,-1.411812,-1.011205,-2.898197,0.191651,2.972993


In [64]:
train_pca['target'] = train['target']
train_pca

,Pc0,Pc1,Pc2,Pc3,Pc4,Pc5,Pc6,Pc7,Pc8,Pc9,...,Pc91,Pc92,Pc93,Pc94,Pc95,Pc96,Pc97,Pc98,Pc99,target
0,6.080757,7.954875,-2.802692,2.653759,4.011957,-1.882538,0.594221,-9.733253,2.039697,4.315181,...,-1.447385,1.617427,-1.351892,1.422038,-0.240246,-0.785429,0.665951,-0.759634,-4.678932,2
1,30.036447,-21.873893,3.636547,-1.850623,1.153098,7.418425,-3.877800,2.782597,0.858048,-6.112473,...,0.232597,-0.208322,0.866749,-1.054738,0.435261,0.155073,0.231425,1.485202,1.096075,2
2,-12.107453,16.221957,22.471516,0.424821,5.054502,-11.044553,-2.081414,0.694179,-6.138139,-1.522114,...,-0.368128,2.776458,1.519931,1.090265,0.409069,1.210661,-0.946403,1.408324,-0.607774,2
3,1.346085,23.551188,-7.554624,-2.560278,6.970243,3.910359,3.918008,-0.263672,-4.631270,-5.018888,...,0.404432,-1.352761,5.270492,6.401115,0.102558,-3.367166,2.121251,2.746955,4.293790,1
4,1.880267,-5.478529,-8.641508,-13.674802,9.698266,3.338851,5.228863,0.741220,-3.902216,-0.839658,...,-1.602891,2.937229,4.845227,-1.596047,3.014279,-0.140397,-0.919787,1.552691,-0.941675,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33617,17.460651,23.549214,22.178911,-4.954198,8.310337,-6.282133,10.369579,7.570798,0.656124,1.357701,...,-4.213453,-5.313589,-6.332882,4.621230,4.585040,0.232472,4.850231,5.790078,-3.050809,2
33618,-30.861525,11.351626,20.171907,-3.803731,-6.453376,-15.332609,1.663190,1.987642,-1.803880,-4.017764,...,-0.902648,3.671061,2.179107,0.026893,-2.669256,-0.447677,-1.557754,1.032013,1.159668,1
33619,-0.303412,24.541428,-6.630254,-14.424322,1.281926,9.405782,-3.889374,5.640531,0.160012,-4.657691,...,0.128933,-7.730116,0.851782,-0.364746,-4.354710,-1.967472,1.234557,-1.556982,-6.643284,1
33620,16.179192,-1.277407,-0.925478,-1.143221,-5.675464,6.141842,16.658610,3.870176,-6.176522,2.710918,...,-1.655715,3.008332,-5.446652,-0.947447,-1.411812,-1.011205,-2.898197,0.191651,2.972993,0


In [65]:
test_pca = data_pca.iloc[33622:,].reset_index(drop = True)
test_pca

,Pc0,Pc1,Pc2,Pc3,Pc4,Pc5,Pc6,Pc7,Pc8,Pc9,...,Pc90,Pc91,Pc92,Pc93,Pc94,Pc95,Pc96,Pc97,Pc98,Pc99
0,-14.813895,7.748919,-9.302643,-6.765887,6.874572,1.883700,16.600385,5.802258,-4.230990,-0.332097,...,1.642781,3.254021,-1.244384,1.966430,-1.959105,2.140955,-2.119048,2.451762,0.931716,-0.777743
1,44.885589,-1.587405,-4.807591,-2.897776,-11.663116,-4.560244,-3.219777,-3.627263,9.421503,6.784828,...,0.298728,1.224055,-0.322493,0.884271,-0.961779,0.016870,0.700811,2.045717,-2.245685,-0.120623
2,-4.442418,-12.178529,1.892988,4.460863,4.467275,0.734715,0.362695,13.477985,-10.531631,2.656666,...,-0.529784,-3.412439,-1.903696,-0.675759,2.022643,0.863083,-1.709486,-2.241390,-0.693918,0.398838
3,-28.733988,-20.219348,-0.345278,-6.885177,17.080621,-3.913046,2.700387,-5.702377,-6.185927,13.942032,...,1.512779,1.918243,1.137098,-0.605504,-0.357081,1.662577,-0.194793,-0.180195,0.193139,2.178271
4,-35.255716,-8.579253,9.641255,8.636222,1.369666,15.507913,5.490041,0.199742,1.618080,-3.300651,...,-2.265785,1.778512,1.237000,-1.062031,-1.509895,1.335415,1.032296,-0.046078,-0.479941,2.420318
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14405,-36.481625,10.091845,-4.454655,4.204408,2.532401,1.721281,-4.020183,1.733923,-3.180866,-5.533264,...,-1.232750,0.909936,-3.070576,1.485871,-3.405902,-4.822651,1.287572,-2.979550,-4.302379,-0.422440
14406,-0.594139,5.332278,-4.769545,7.976711,-0.962376,0.968390,3.429993,-7.823931,-2.015112,-1.255054,...,-0.441158,-1.793284,3.252938,-1.587510,-3.216585,-1.404560,2.865136,0.914498,-1.215238,1.628349
14407,-0.671166,17.183658,-7.553657,2.413915,-4.123622,4.016218,-6.320733,-6.179823,-3.477777,-0.656162,...,0.929953,0.170927,-0.476589,1.865272,-1.023658,-0.539059,-1.320524,0.588805,-0.588420,-3.756595
14408,-11.658364,9.839462,22.364111,2.269509,-3.218073,-14.177568,-0.873510,1.288244,-8.167138,-5.689238,...,-1.929132,1.060163,1.407730,-1.320607,-2.343418,-1.944141,0.975199,1.406309,-0.487667,-1.250202


In [33]:
X_train, X_test, y_train, y_test = train_test_split(train.drop('target', axis=1), train['target'],
                                                    test_size = 0.2,random_state = 10)

## KNN

In [34]:
from sklearn.neighbors import KNeighborsClassifier

In [35]:
knn = KNeighborsClassifier(n_neighbors=10)
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("Accuracy : {}".format(round(accuracy_score(y_test, y_pred), 3)))

Accuracy : 0.399


## Decision Tree

In [36]:
tree = DecisionTreeClassifier()  
tree.fit(X_train, y_train)
y_pred = tree.predict(X_test)
print("Accuracy : {}".format(round(accuracy_score(y_test, y_pred), 3)))

Accuracy : 0.366


## Random Forest

In [96]:
RFC = RandomForestClassifier()
RFC.fit(X_train, y_train)
y_pred = RFC.predict(X_test)
print("Accuracy : {}".format(round(accuracy_score(y_test, y_pred), 3)))

Accuracy : 0.335


## XGboost

In [37]:
import xgboost as xgb

In [38]:
xgb_classifier = xgb.XGBClassifier(n_estimators=100, tree_method='hist', eta=0.1, max_depth=3)

xgb_classifier.fit(X_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False, eta=0.1,
              eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
              importance_type=None, interaction_constraints='',
              learning_rate=0.100000001, max_bin=256, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=3, max_leaves=0, min_child_weight=1,
              missing=nan, monotone_constraints='()', n_estimators=100,
              n_jobs=0, num_parallel_tree=1, objective='multi:softprob',
              predictor='auto', random_state=0, ...)

In [39]:
y_pred = xgb_classifier.predict(X_test)
print("Accuracy : {}".format(round(accuracy_score(y_test, y_pred), 3)))

Accuracy : 0.402


## SVM

In [97]:
from sklearn import svm

In [ ]:
#Create a svm Classifier
clf = svm.SVC(kernel='linear') # Linear Kernel

#Train the model using the training sets
clf.fit(X_train, y_train)

#Predict the response for test dataset
y_pred = clf.predict(X_test)

In [ ]:
print("Accuracy : {}".format(round(accuracy_score(y_test, y_pred), 3)))

## Naive Bayes

In [40]:
from sklearn.naive_bayes import GaussianNB

In [41]:
gnb = GaussianNB()
gnb.fit(X_train, y_train)
y_pred = gnb.predict(X_test)

In [42]:
print("Accuracy : {}".format(round(accuracy_score(y_test, y_pred), 3)))

Accuracy : 0.398


## Neural Network

In [43]:
import keras
from keras.models import Sequential,Model
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D, Conv1D, MaxPooling1D, Flatten
from keras.layers import Bidirectional, LSTM, Normalization

In [45]:
batch_size = 32
epochs = 20
num_classes = 3

In [46]:
data_training = pd.read_csv(train_dataset)

In [47]:
Y_train = data_training[["biological_process","cellular_component","molecular_function"]]
Y_train

,biological_process,cellular_component,molecular_function
0,0,0,1
1,0,0,1
2,0,0,1
3,0,1,0
4,0,0,1
...,...,...,...
33617,0,0,1
33618,0,1,0
33619,0,1,0
33620,1,0,0


In [56]:
X_train, X_test, y_train, y_test = train_test_split(train.drop('target', axis=1), Y_train,
                                                    test_size = 0.1,random_state = 10)

In [57]:
model = Sequential()
# model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

In [ ]:
model = Sequential()
model.add(Conv1D(16,3,activation='relu', input_shape=(1280,1)))
model.add(MaxPooling1D(2))
model.add(Conv1D(32,3,activation='relu'))
model.add(MaxPooling1D(2))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Normalization())
model.add(Dropout(0.2))
model.add(Dense(32, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

In [48]:
from keras.callbacks import ReduceLROnPlateau, EarlyStopping

early_stopper = EarlyStopping( monitor = 'val_acc' , min_delta = 0.0005, patience = 3 )
 
reduce_lr = ReduceLROnPlateau( monitor = 'val_loss' , patience = 2 , cooldown = 0)
 
callbacks = [ reduce_lr , early_stopper]

In [58]:
model.compile(loss=keras.losses.categorical_crossentropy, optimizer=keras.optimizers.Adam(),
              metrics=['accuracy'])

In [59]:
history = model.fit(X_train, y_train, epochs=epochs,verbose=1, validation_split = 0.1)

Epoch 1/20
852/852 [==============================] - 5s 5ms/step - loss: 1.0865 - accuracy: 0.3823 - val_loss: 1.0654 - val_accuracy: 0.4184
Epoch 2/20
852/852 [==============================] - 4s 4ms/step - loss: 1.0694 - accuracy: 0.4074 - val_loss: 1.0610 - val_accuracy: 0.4144
Epoch 3/20
852/852 [==============================] - 4s 4ms/step - loss: 1.0657 - accuracy: 0.4058 - val_loss: 1.0591 - val_accuracy: 0.4121
Epoch 4/20
852/852 [==============================] - 4s 4ms/step - loss: 1.0623 - accuracy: 0.4076 - val_loss: 1.0560 - val_accuracy: 0.4121
Epoch 5/20
852/852 [==============================] - 4s 4ms/step - loss: 1.0594 - accuracy: 0.4138 - val_loss: 1.0549 - val_accuracy: 0.4174
Epoch 6/20
852/852 [==============================] - 4s 4ms/step - loss: 1.0574 - accuracy: 0.4091 - val_loss: 1.0600 - val_accuracy: 0.4157
Epoch 7/20
852/852 [==============================] - 4s 5ms/step - loss: 1.0679 - accuracy: 0.4018 - val_loss: 1.0629 - val_accuracy: 0.4078
Epoch 

In [60]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_6 (Dense)             (None, 256)               327936    
                                                                 
 dropout_3 (Dropout)         (None, 256)               0         
                                                                 
 dense_7 (Dense)             (None, 256)               65792     
                                                                 
 dropout_4 (Dropout)         (None, 256)               0         
                                                                 
 dense_8 (Dense)             (None, 128)               32896     
                                                                 
 dropout_5 (Dropout)         (None, 128)               0         
                                                                 
 dense_9 (Dense)             (None, 64)               

In [61]:
test_eval = model.evaluate(X_test, y_test, verbose=0)
print('Test loss:', test_eval[0])
print('Test accuracy:', test_eval[1])

Test loss: 1.0636762380599976
Test accuracy: 0.401724636554718


In [69]:
pred_val = model.predict(test)

451/451 [==============================] - 1s 2ms/step


In [70]:
pred_val

array([[0.3360189 , 0.2993824 , 0.3645987 ],
       [0.36925137, 0.13977505, 0.4909736 ],
       [0.35122177, 0.5818949 , 0.06688334],
       ...,
       [0.35012922, 0.23925881, 0.410612  ],
       [0.356827  , 0.39359424, 0.24957874],
       [0.34281537, 0.26586333, 0.39132127]], dtype=float32)

In [71]:
pred_val = np.argmax(pred_val, axis=1)

In [72]:
pred_val

array([2, 2, 1, ..., 2, 1, 2])

## Submission

In [66]:
sub = pd.read_csv('/kaggle/input/instadeep-challenge-soai/archive/submissionSample.csv')
sub.head()

,ID,target
0,test_0,cellular_component
1,test_1,biological_process
2,test_2,cellular_component
3,test_3,cellular_component
4,test_4,cellular_component


In [67]:
pred_val = xgb_classifier.predict(test)

In [73]:
sub['target'] = pred_val
sub.head()

,ID,target
0,test_0,2
1,test_1,2
2,test_2,1
3,test_3,2
4,test_4,2


In [74]:
sub['target'] = sub['target'].replace([0,1,2],["biological_process","cellular_component","molecular_function"]).values

In [75]:
sub.set_index('ID', inplace = True)

In [76]:
sub

,target
ID,
test_0,molecular_function
test_1,molecular_function
test_2,cellular_component
test_3,molecular_function
test_4,molecular_function
...,...
test_14405,molecular_function
test_14406,molecular_function
test_14407,molecular_function


In [79]:
sub.to_csv("submission_protein_9.csv")